# SoleXplorer

### The Swiss Army knife for machine learning

In [1]:
using SoleXplorer

  Activating project at `~/Documents/Aclai/Sole/SoleXplorer.jl`
[ Info: Running `conda config --set pip_interop_enabled true --file /home/paso/.julia/conda/3/x86_64/condarc-julia.yml` in root environment
[ Info: Running `pip install git+https://github.com/jobregon1212/rulecosi.git` in root environment


  Cloning https://github.com/jobregon1212/rulecosi.git to /tmp/pip-req-build-k3kgz0t8


  Running command git clone --filter=blob:none --quiet https://github.com/jobregon1212/rulecosi.git /tmp/pip-req-build-k3kgz0t8


  Resolved https://github.com/jobregon1212/rulecosi.git to commit cd7acee23c9c2f6e7f433aafac7bab446b67f3d2
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'


[ Info: Running `pip install scikit-learn` in root environment


Load NATOPS dataset, composed of time series

In [2]:
X, y = load_arff_dataset("NATOPS")

(360×24 DataFrame
 Row │ X[Hand tip l]                      Y[Hand tip l]                      Z ⋯
     │ Array…                             Array…                             A ⋯
─────┼──────────────────────────────────────────────────────────────────────────
   1 │ [-0.519771, -0.52758, -0.531415,…  [-2.14011, -2.18043, -2.18425, -…  [ ⋯
   2 │ [-0.489753, -0.48607, -0.484529,…  [-1.55293, -1.54966, -1.55206, -…  [
   3 │ [-0.521346, -0.518394, -0.522321…  [-1.72326, -1.72407, -1.72326, -…  [
   4 │ [-0.57022, -0.562064, -0.565967,…  [-1.91196, -1.90369, -1.90527, -…  [
   5 │ [-0.624417, -0.626031, -0.625388…  [-1.84287, -1.84026, -1.84688, -…  [ ⋯
   6 │ [-0.502501, -0.502525, -0.499415…  [-2.17556, -2.15613, -2.18516, -…  [
   7 │ [-0.488461, -0.489463, -0.487539…  [-2.17242, -2.18203, -2.18057, -…  [
   8 │ [-0.468105, -0.410602, -0.473909…  [-1.86535, -1.89011, -1.87105, -…  [
  ⋮  │                 ⋮                                  ⋮                    ⋱
 354 │ [-0.500404, -0.

### downsize dataset
it is important to downsize the dataset to avoid long running times and to avoid memory issues

In [ ]:
using StatsBase: sample
num_cols_to_sample, num_rows_to_sample, rng = 10, 50, Xoshiro(11)
chosen_cols = sample(rng, 1:size(X, 2), num_cols_to_sample; replace=false)
chosen_rows = sample(rng, 1:size(X, 1), num_rows_to_sample; replace=false)
X = X[chosen_rows, chosen_cols]
y = y[chosen_rows]

Xoshiro(0x0991231718e930cb, 0x28e1460087a5d0ff, 0x4d62c780da1946f0, 0x764f51fefd621192, 0x434e1895e0078176)

### Let's start diving into available models
## Decision Tree

In [ ]:
model = symbolic_analysis(X, y; models=(type=:decisiontree,), preprocess=(;rng))

For reproducible experiments, always include 'preprocess=(;rng)' in your 'preprocess' configuration.

# Utility Functions

## prepare_dataset
if it's not needed to train a model, but only prepare dataset for future analisys, prepare_dataset should be used.
Will return a Modelset structure.

Please remember that proper dataset preparation is model dependent, expecially modal experiments need ad hoc dataset preparation.

### Example 1:
No extra parameters passed: it will be used default model: decision tree.

In [2]:
no_parameters = prepare_dataset(X, y)

┌ Error: Failed to revise /home/paso/Documents/Aclai/Sole/SoleXplorer.jl/src/interface.jl
│   exception = Revise.ReviseEvalException("/home/paso/Documents/Aclai/Sole/SoleXplorer.jl/none:0", ErrorException("invalid redefinition of constant SoleXplorer.DatasetInfo"), Any[(top-level scope at none:0, 1)])
└ @ Revise /home/paso/.julia/packages/Revise/mLfYT/src/packagedef.jl:776
┌ Warning: The running code does not match the saved version for the following files:
│ 
│   /home/paso/Documents/Aclai/Sole/SoleXplorer.jl/src/interface.jl
│ 
│ If the error was due to evaluation order, it can sometimes be resolved by calling `Revise.retry()`.
│ Use Revise.errors() to report errors again. Only the first error in each file is shown.
│ Your prompt color may be yellow until the errors are resolved.
└ @ Revise /home/paso/.julia/packages/Revise/mLfYT/src/packagedef.jl:893


UndefVarError: UndefVarError: `X` not defined in `Main`
Suggestion: check for spelling errors or missing imports.
Hint: a global variable of this name may be made accessible by importing DimensionalData in the current active module Main

### Example 2:
model type passed: dataset will be prepared for modal tasks.

In [ ]:
model_type = prepare_dataset(X, y; model=(type=:modaldecisiontree,))

### Example 3:
Also pass parameters specific to model choosen.

In [ ]:
parametrized_model_type = prepare_dataset(X, y; 
model=(type=:xgboost,
        params=(
            num_round=20, 
            booster="gbtree", 
            eta=0.5,
            num_parallel_tree=10, 
            max_depth=8, 
        )
    )
)